In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
#nothing in this file seems to work properly
df = gpd.read_file('./fire21_2.gdb/')

In [8]:
dataB = df
print(df.columns.tolist())
dataB.columns

['YEAR_', 'STATE', 'AGENCY', 'UNIT_ID', 'FIRE_NAME', 'INC_NUM', 'ALARM_DATE', 'CONT_DATE', 'CAUSE', 'COMMENTS', 'REPORT_AC', 'GIS_ACRES', 'C_METHOD', 'OBJECTIVE', 'FIRE_NUM', 'Shape_Length', 'Shape_Area', 'COMPLEX_NAME', 'COMPLEX_INCNUM', 'geometry']


Index(['YEAR_', 'STATE', 'AGENCY', 'UNIT_ID', 'FIRE_NAME', 'INC_NUM',
       'ALARM_DATE', 'CONT_DATE', 'CAUSE', 'COMMENTS', 'REPORT_AC',
       'GIS_ACRES', 'C_METHOD', 'OBJECTIVE', 'FIRE_NUM', 'Shape_Length',
       'Shape_Area', 'COMPLEX_NAME', 'COMPLEX_INCNUM', 'geometry'],
      dtype='object')

In [9]:
dataB = dataB.set_crs(epsg = 3310)
dataB = dataB.to_crs(epsg = 4326)
dataB.geometry.bounds

,minx,miny,maxx,maxy
0,-121.355040,38.886106,-121.340169,38.890478
1,-121.387747,38.815755,-121.351426,38.837968
2,-121.338750,38.841884,-121.331711,38.845649
3,-121.275792,38.960930,-121.270471,38.963578
4,-121.302496,39.485433,-121.298675,39.487215
...,...,...,...,...
21683,-123.009418,40.846974,-123.003509,40.850995
21684,-119.569151,37.516216,-119.545889,37.529345
21685,-122.147823,40.401240,-122.136553,40.410468
21686,-123.056564,40.845766,-123.051091,40.850576


Remove fires not in California and before 1950

In [10]:
dataB.set_index('STATE' , inplace = True) #run entire cell only once after reading from above cell

print('entries:', len(dataB))
dataB.drop(index = 'NV' , axis = 0, inplace = True) 
dataB.drop(index = 'OR' , axis = 0, inplace = True) 
dataB.drop(index = 'AZ' , axis = 0 , inplace = True)
print('entries:', len(dataB))

# dataB = dataB[dataB['STATE'] != 'AZ'] alternate method of cleaning

dataB.reset_index('STATE' , inplace = True)

dataB.set_index('YEAR_' , inplace = True)

# F_before_1950 = dataB[dataB.index < 1950]
# print(F_before_1950)
for year in range(1950): #range of known years provided in csv we need removed
    if (year in dataB.index):
        dataB.drop(index = year , axis = 0 , inplace = True)
print('entries:', len(dataB))
dataB.reset_index('YEAR_' , inplace = True)

entries: 21688
entries: 21593
entries: 21593


remove entries with missing year

In [15]:
print(len(dataB))
print(dataB['YEAR_'].isna().sum()) #isna() is bool but python is 'ducklike' and can add with .sum()
dataB = dataB[dataB['YEAR_'].notna()] #reassigns dataframe to the same datafram with column year with no missing year entries
print(len(dataB))

print(dataB[dataB['YEAR_'] == '1950']) #print all years at 1950

21525
0
21525
      YEAR_ STATE AGENCY UNIT_ID        FIRE_NAME   INC_NUM  \
1078   1950    CA    CDF     SNU      P.G.& E. #4  00000000   
1079   1950    CA    CDF     LMU    BAGGETT GULCH  00000000   
2237   1950    CA    CDF     MVU          CONEJOS  00000000   
2238   1950    CA    USF     CNF                   00000000   
2423   1950    CA    CDF     MVU  SYCAMORE CANYON  00000000   
...     ...   ...    ...     ...              ...       ...   
18617  1950    CA    NPS     KNP         GOAT MTN  00000000   
18618  1950    CA    NPS     KNP        EAGLE PEA  00000000   
18619  1950    CA    NPS     KNP        DEER COVE  00000000   
18620  1950    CA    NPS     KNP        LIGHTN CR  00000000   
18752  1950    CA    NPS     YNP                       None   

                      ALARM_DATE                  CONT_DATE  CAUSE  COMMENTS  \
1078   1950-09-03T00:00:00+00:00                       None   14.0             
1079   1950-06-02T00:00:00+00:00                       None    9.0   

Find broken index for datetime (nearly impossible due to ALARM_DATE format)

In [13]:

# dataB[dataB.index == '219-05-29 00:00:00']

dataB.ALARM_DATE.dtype

dtype('O')

Sort Chronologically

In [45]:
dataB.set_index('YEAR_' , inplace = True)
dataB.sort_values(by = 'YEAR_' , ascending = True , inplace = True)
dataB.reset_index('YEAR_' , inplace = True)

dataB['datetime'] = pd.to_datetime(dataB['ALARM_DATE'] , errors = 'coerce') #create new datetime column
dataB.set_index('datetime' , inplace = True) #set new column to index
dataB.sort_values(by='datetime', ascending = True, inplace = True) #sort values
dataB.reset_index('datetime' , inplace = True) #reset index to numbered list
dataB.drop('datetime' , axis = 1 , inplace = True) #delete the datetime column

print(len(dataB) , dataB.index)
dataB.tail

# for i in range(8):
#     print(dataB[dataB.index == i*1000].ALARM_DATE)

21525 RangeIndex(start=0, stop=21525, step=1)


<bound method NDFrame.tail of       YEAR_ STATE AGENCY UNIT_ID     FIRE_NAME   INC_NUM  \
0      1898    CA    CCO     VNC      MATILIJA  00000000   
1      1898    CA    CCO     VNC     COZY DELL  00000000   
2      1898    CA    CCO     VNC    LOS PADRES  00000000   
3      1902    CA    CCO     VNC        FEROUD  00000000   
4      1903    CA    CCO     VNC   SAN ANTONIO  00000000   
...     ...   ...    ...     ...           ...       ...   
21520  2019    CA    USF     TNF        BORDER  00030785   
21521  2019    CA    USF     INF         KELTY  00001465   
21522  2019    CA    USF     HTF      TAMARACK  00030272   
21523  2019    CA    CCO     ORC  BLUE DIAMOND  19059616   
21524  2019    CA    NPS     MNP          STAR  00013598   

                      ALARM_DATE                  CONT_DATE  CAUSE  \
0      1898-04-01T00:00:00+00:00                       None   14.0   
1      1898-09-01T00:00:00+00:00                       None   14.0   
2      1898-09-01T00:00:00+00:00       